In [1]:
import pandas as pd

In [2]:
df = {}
for k in ["train", "val", "test"]:
    df[k] = {}
    for c in ["data", "bong"]:
        df[k][c] = pd.read_excel(f"setting/{c}_v2_{k}.xlsx", index_col=0)

In [9]:
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier

In [3]:
X_train = df["train"]["bong"]
X_val = df["val"]["bong"]
X_test = df["test"]["bong"]

y_train = (df["train"]["data"]["Final decision"] == "INCLUDE").apply(int)
y_val = (df["val"]["data"]["Final decision"] == "INCLUDE").apply(int)
y_test = (df["test"]["data"]["Final decision"] == "INCLUDE").apply(int)

In [7]:
new_best_features = X_train.columns
new_best_features

Index(['bong_1', 'bong_2', 'bong_3', 'bong_4', 'bong_5', 'bong_6', 'bong_7',
       'bong_8', 'bong_9', 'bong_10',
       ...
       'bong_991', 'bong_992', 'bong_993', 'bong_994', 'bong_995', 'bong_996',
       'bong_997', 'bong_998', 'bong_999', 'bong_1000'],
      dtype='object', length=1000)

In [10]:
clf = DecisionTreeClassifier(random_state=42, max_depth=5, min_impurity_decrease=0.0)
clf.fit(X_train[new_best_features], y_train)
print(clf.score(X_train[new_best_features], y_train))
print(classification_report(y_test, clf.predict(X_test[new_best_features]), digits=4))

0.7671428571428571
              precision    recall  f1-score   support

           0     0.6488    0.5798    0.6124       188
           1     0.6595    0.7217    0.6892       212

    accuracy                         0.6550       400
   macro avg     0.6541    0.6507    0.6508       400
weighted avg     0.6545    0.6550    0.6531       400



In [7]:
from matplotlib import pyplot as plt
from sklearn import tree

In [8]:
fig, ax = plt.subplots(1, 1, figsize=(265, 85))
tree.plot_tree(clf, feature_names=new_best_features, ax=ax, fontsize=45, class_names=["Exclude","Include"]);

In [8]:
len(clf.feature_names_in_[[i for i in clf.tree_.feature if i>=0]])

26

In [31]:
y_pred_train = clf.predict(X_train[new_best_features])

In [13]:
ix=323
label = y_train.loc[ix]
row = X_train.loc[ix]
row["b5(Y)"]

0.9979381

In [10]:
for x in new_best_features:
    if "(N)" in x:
        print(x, row[x])

lf5(N) 0.0057145
b12_v1_v3(N) 0.97937953
raw10(N) 0.00728041
exp1_v1(N) 0.00385545
raw66_v3(N) 0.00576267
raw45_v3(N) 0.00718782
raw48(N) 0.00592829
raw70_v2(N) 0.00502992
exp16_v1(N) 0.00394058
raw71(N) 0.00645994
lf6(N) 0.00528816
raw21(N) 0.00557318
lf18(N) 0.00407362
raw51(N) 0.0140639
exp1_v2(N) 0.00459745
raw41(N) 0.00524803
b7(N) 0.00623979
raw43(N) 0.00437084
b6_v1_v2(N) 0.00404352
raw30(N) 0.00616835
b13_v2(N) 0.99148273
raw70_v3(N) 0.00682619
exp25(N) 0.00428802
raw26_v3(N) 0.00609419
exp22_v1(N) 0.00867163
exp21_v3(N) 0.00572083
exp2(N) 0.00429618
b4_v1(N) 0.00430295
exp12_v2(N) 0.01082657
b6_v1(N) 0.00464466
raw80(N) 0.00633835
raw62(N) 0.00425419
b6(N) 0.00400467
raw73_v2(N) 0.00457074
b2_v1(N) 0.00443125
b12_v3(N) 0.971923
raw77(N) 0.01284178
raw49_v2(N) 0.00605363
raw31_v2(N) 0.00645046
raw25(N) 0.00668056
raw26_v1(N) 0.00611222
raw11(N) 0.00686021
raw69_v2(N) 0.01471701
raw1(N) 0.00601656
raw70_v1(N) 0.0064229
exp10_v3(N) 0.00700678
exp1_v3_v2(N) 0.00477606
raw63_v1(N) 

In [32]:
des = [
    ("exp1_v3_v3(Y)", 0.993, False), #(NLLF) Q: Does the abstract address the relationship between agroecological practices and climate change? A: Maybe no
    ("convent*", 0.5, True), #(LF) The abstract does not have a word with the prefix convent
    ("model", 0.5, True), #(LF) The abstract does not have the word model
    ("raw76(N)", 0.856, False), #(NLLF) Q: Does the abstract discuss agroecology’s benefits for ecosystem services? A: Maybe yes
    ("raw50(N)", 0.005, False), #(NLLF) Q: Does the abstract discuss certified organic production? A: Yes
    # ("b12_v1_v2(N)", 0.997, False), #(NLLF) Q: Does the abstract analyze how agroecology affects nitrogen dynamics? A: Maybe yes
    # ("raw73_v3(Y)", 0.996, False)
    # ("b6(N)", 0.061, True) #(NLLF) Q: Does the abstract assess agroecological practices' impact on climate change? A: Yes
#     ("exp1_v3(Y)", 0.736, True),
#     ("grass", 0.5, True),
#     ("agroforestry", 1.5, True),
]
for i, ix in enumerate(X_train.index):
    label = y_train.loc[ix]
    row = X_train.loc[ix]
    if label == y_pred_train[i]:
        if all([(row[c] <= t) == b for (c, t, b) in des]):
            print("Title:", df["train"]["data"].loc[ix]["Article Title"])
            print("Abstract:", df["train"]["data"].loc[ix]["Abstract (AB)"])
            print(ix, label)

Title: Does carbon farming provide a cost-effective option to mitigate GHG emissions? Evidence from China
Abstract: In this study, we apply a whole farm bioeconomic analysis to explore the changes in land use, farm practices and on-farm greenhouse gas (GHG) emission under varying levels of agricultural greenhouse gas abatement incentives in the form of a carbon tax for a semi-arid crop-livestock farming system in China's Loess Plateau. Our results show that the optimised agricultural enterprises move towards being cropping-dominated reducing on-farm emission since livestock perform is the major source of emission. Farmers employ less oats-based and rapeseed-based rotations but more dry pea-based rotations in the optimal enterprise mix. A substantial reduction in on-farm greenhouse gas emission can be achieved at low cost with a small increase in carbon incentives. Our estimates indicate that crop-livestock farmers in China's Loess Plateau may reduce their on-farm GHG emission between 1

In [16]:
from sklearn.dummy import DummyClassifier

In [17]:
dummy = DummyClassifier(strategy="uniform", random_state=42)
dummy.fit(X_train[new_best_features], y_train)
print(dummy.score(X_train[new_best_features], y_train))
print(classification_report(y_test, dummy.predict(X_test[new_best_features]), digits=4))

0.5064285714285715
              precision    recall  f1-score   support

           0     0.4824    0.5106    0.4961       188
           1     0.5423    0.5142    0.5278       212

    accuracy                         0.5125       400
   macro avg     0.5124    0.5124    0.5120       400
weighted avg     0.5141    0.5125    0.5129       400



In [18]:
clf = DecisionTreeClassifier(random_state=42, max_depth=8)
clf.fit(X_train[new_best_features], y_train)
print(clf.score(X_train[new_best_features], y_train))
print(classification_report(y_test, clf.predict(X_test[new_best_features]), digits=4))
set(clf.feature_names_in_[[i for i in clf.tree_.feature if i>=0]])

0.8678571428571429
              precision    recall  f1-score   support

           0     0.6333    0.7074    0.6683       188
           1     0.7105    0.6368    0.6716       212

    accuracy                         0.6700       400
   macro avg     0.6719    0.6721    0.6700       400
weighted avg     0.6742    0.6700    0.6701       400



{'agriculture',
 'agroforestry',
 'agroforestry systems',
 'b11(N)',
 'b12(N)',
 'b12_v1_v2(N)',
 'b13_v3(N)',
 'b3(N)',
 'b4_v1(N)',
 'b5(Y)',
 'b8(Y)',
 'b8_v1(Y)',
 'b8_v2(Y)',
 'b9_v1(N)',
 'b9_v3(Y)',
 'convent*',
 'crop',
 'exp12_v1(N)',
 'exp14_v2(Y)',
 'exp17_v2(N)',
 'exp19_v2(N)',
 'exp1_v1_v3(N)',
 'exp1_v2(N)',
 'exp1_v3(Y)',
 'exp1_v3_v2(Y)',
 'exp1_v3_v3(Y)',
 'exp21_v1(Y)',
 'exp22_v2_v1(Y)',
 'exp22_v2_v2(N)',
 'exp22_v2_v2(Y)',
 'exp27(Y)',
 'exp28(N)',
 'exp30(Y)',
 'exp33_v3(N)',
 'exp36(N)',
 'exp5_v1(Y)',
 'exp8(Y)',
 'grass',
 'impact of',
 'lf1(N)',
 'lf1(Y)',
 'lf10(N)',
 'lf17(Y)',
 'lf19(N)',
 'lf5(Y)',
 'lf9(N)',
 'meta-analysis',
 'model',
 'organic',
 'predict',
 'raw10(N)',
 'raw21(N)',
 'raw36(Y)',
 'raw39(Y)',
 'raw42(Y)',
 'raw49_v3(N)',
 'raw50(N)',
 'raw52(N)',
 'raw58(Y)',
 'raw59(N)',
 'raw62(N)',
 'raw62(Y)',
 'raw65(Y)',
 'raw66(N)',
 'raw67(N)',
 'raw69_v2(N)',
 'raw70_v2(N)',
 'raw73_v2(Y)',
 'raw73_v3(Y)',
 'raw74_v1(N)',
 'raw76(N)',
 'raw79(Y